# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## 1.Start by creating a new Notebook for this assignment

In [ ]:
#step1. import standard libraries
import pandas as pd
import numpy as np
import requests

In [ ]:
#ste21. import bsp
!pip install beautifulsoup4
    

In [ ]:
from bs4 import BeautifulSoup

## 2.Use the Notebook to build the code to scrape the following Wikipedia page...

In [ ]:
# Step2. Get data from Wikipedia
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page= requests.get(wiki_link).text
wiki_soup = BeautifulSoup(wiki_page,'xml')
#print (wiki_soup)

In [ ]:
#Step3. Get row data

#Step3.1: Get row data table
table = wiki_soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

#Step 3.2: Clean row data data
for tr_cell in table.find_all('tr'):#Iterating through rows and columns
    
    counter = 1
    PC     = -1
    Brgh = -1
    Nbh = -1
    
    for td_cell in tr_cell.find_all('td'): #Iterating through rows and columns
        if counter == 1: 
            PC = td_cell.text
        if counter == 2: 
            Brgh = td_cell.text
            checkBorough = td_cell.find('a')
            
        if counter == 3: 
            Nbh = str(td_cell.text).strip()
            checkNbh = td_cell.find('a')
            
        counter +=1
        
    if (PC== 'Not assigned' or Brgh == 'Not assigned' or Nbh == 'Not assigned'): 
        continue
    try:
        if ((checkBorough is None) or (checkNbh is None)):
            continue
    except:
        pass
    if(PC == -1 or Brgh == -1 or Nbh == -1):
        continue
        
    Postcode.append(PC)
    Borough.append(Brgh)
    Neighbourhood.append(Nbh)

In [ ]:
#More than one neighborhood can exist in one postal code area.
#For example, in the table on the Wikipedia page, you will notice that M5A
#is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown
#in row 11 in the above table.

#Step4. Get postal code with more than 1 neighbourhood

PC_unique = set(Postcode)

UniquePostcode     = []
UniqueBorough    = []
UniqueNeighbourhood = []

for pc_uniqueGD in PC_unique:
    pcv = ''; brgv = ''; nghv = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if pc_uniqueGD == postcode_element:
            pcv = postcode_element;
            brgv = Borough[postcode_idx]
            if nghv == '': 
                nghv = Neighbourhood[postcode_idx]
            else:
                nghv = nghv + ', ' + Neighbourhood[postcode_idx]
    UniquePostcode.append(pcv)
    UniqueBorough.append(brgv)
    UniqueNeighbourhood.append(nghv)

#Step5. Get clean pd.dataframe from row data from wikipedia
TableData= {'PostalCode': UniquePostcode, 'Borough': UniqueBorough, 'Neighbourhood': UniqueNeighbourhood}
df_TableData = pd.DataFrame.from_dict(TableData)
df_TableData.head(16)

In [ ]:
df_TableData.shape